---
title: "ANOVA de Una Vía en Python"
author: "Traducido y adaptado de R por un asistente de IA"
format: live-html
pyodide: 
    packages: 
        - numpy
        - pandas
        - matplotlib
        - seaborn
        - pingouin
        - statsmodels
---

## ¿Qué es el ANOVA de Una Vía?

El **Análisis de Varianza de una vía** (One-way ANOVA) es una extensión de la prueba t independiente para comparar medias cuando tenemos **tres o más grupos**. Es la versión paramétrica del test de Kruskal-Wallis.

### Características del ANOVA

- **Compara medias**: Evalúa si las medias de varios grupos son estadísticamente diferentes
- **Un solo factor**: Analiza el efecto de una sola variable independiente (factor)
- **Múltiples grupos**: Funciona con 3 o más grupos (con 2 grupos es equivalente a t-test)

### Principio del ANOVA

El ANOVA compara dos tipos de variabilidad:
1. **Variabilidad entre grupos**: ¿Qué tan diferentes son las medias de los grupos?
2. **Variabilidad dentro de grupos**: ¿Qué tan variables son los datos dentro de cada grupo?

Si la variabilidad entre grupos es mucho mayor que la variabilidad dentro de grupos, concluimos que hay diferencias significativas.

## Hipótesis del ANOVA

- **H₀**: Todas las medias poblacionales son iguales (μ₁ = μ₂ = μ₃ = ... = μₖ)
- **H₁**: Al menos una media poblacional es diferente

## Supuestos del ANOVA

1. **Independencia**: Las observaciones deben ser independientes
2. **Normalidad**: Los datos deben seguir una distribución normal en cada grupo
3. **Homogeneidad de varianzas**: Las varianzas deben ser iguales entre grupos (homocedasticidad)

## Ejemplo Práctico: Crecimiento de Plantas

Usaremos un conjunto de datos sobre el crecimiento de plantas bajo diferentes tratamientos (control, tratamiento 1, tratamiento 2).

### 1. Cargar librerías y datos

```{pyodide}
#| echo: false
#| output: false
import micropip
await micropip.install(['pandas', 'numpy', 'scipy', 'seaborn', 'matplotlib', 'pingouin', 'statsmodels'])
```

```{pyodide}
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Crear datos similares al dataset PlantGrowth de R
np.random.seed(123)

# Simular datos de crecimiento de plantas
ctrl = np.random.normal(5.0, 0.6, 10)      # Grupo control
trt1 = np.random.normal(4.7, 0.8, 10)      # Tratamiento 1
trt2 = np.random.normal(5.5, 0.4, 10)      # Tratamiento 2

# Crear DataFrame
my_data = pd.DataFrame({
    'weight': np.concatenate([ctrl, trt1, trt2]),
    'group': ['ctrl'] * 10 + ['trt1'] * 10 + ['trt2'] * 10
})

# Convertir group a categórica con orden específico
my_data['group'] = pd.Categorical(my_data['group'], 
                                 categories=['ctrl', 'trt1', 'trt2'], 
                                 ordered=True)

print("Dataset de crecimiento de plantas:")
print(my_data.head(10))
print(f"\nEstructura del dataset:")
print(f"Total de observaciones: {len(my_data)}")
print(f"Grupos: {my_data['group'].cat.categories.tolist()}")
print(f"Observaciones por grupo: {my_data['group'].value_counts().sort_index()}")
```

### 2. Exploración de Datos

```{pyodide}
# Estadísticas descriptivas por grupo
estadisticas = my_data.groupby('group')['weight'].agg([
    ('count', 'count'),
    ('media', 'mean'),
    ('desv_std', 'std'),
    ('min', 'min'),
    ('max', 'max')
]).round(4)

print("Estadísticas descriptivas por grupo:")
print(estadisticas)
```

### 3. Visualización de los Datos

```{pyodide}
# Crear múltiples visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Boxplot
sns.boxplot(x='group', y='weight', data=my_data, 
            palette=['#00AFBB', '#E7B800', '#FC4E07'], ax=axes[0,0])
axes[0,0].set_title('Distribución por Grupo (Boxplot)')
axes[0,0].set_xlabel('Tratamiento')
axes[0,0].set_ylabel('Peso')

# Violin plot
sns.violinplot(x='group', y='weight', data=my_data, 
               palette=['#00AFBB', '#E7B800', '#FC4E07'], ax=axes[0,1])
axes[0,1].set_title('Distribución por Grupo (Violin Plot)')
axes[0,1].set_xlabel('Tratamiento')
axes[0,1].set_ylabel('Peso')

# Stripplot (puntos individuales)
sns.stripplot(x='group', y='weight', data=my_data, 
              palette=['#00AFBB', '#E7B800', '#FC4E07'], 
              size=8, jitter=True, ax=axes[1,0])
axes[1,0].set_title('Datos Individuales')
axes[1,0].set_xlabel('Tratamiento')
axes[1,0].set_ylabel('Peso')

# Gráfico de medias con error estándar
medias = my_data.groupby('group')['weight'].agg(['mean', 'sem'])
x_pos = range(len(medias))
axes[1,1].errorbar(x_pos, medias['mean'], yerr=medias['sem'], 
                   fmt='o', capsize=5, capthick=2, markersize=8,
                   color=['#00AFBB', '#E7B800', '#FC4E07'])
axes[1,1].set_xticks(x_pos)
axes[1,1].set_xticklabels(medias.index)
axes[1,1].set_title('Medias con Error Estándar')
axes[1,1].set_xlabel('Tratamiento')
axes[1,1].set_ylabel('Peso Promedio')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
```

### 4. Verificación de Supuestos

#### 4.1 Prueba de Normalidad

```{pyodide}
print("Verificación de Supuestos del ANOVA")
print("="*50)
print("\n1. NORMALIDAD (Prueba de Shapiro-Wilk):")
print("H₀: Los datos siguen una distribución normal")
print("H₁: Los datos no siguen una distribución normal\n")

normalidad_ok = True
for grupo in my_data['group'].cat.categories:
    grupo_data = my_data[my_data['group'] == grupo]['weight']
    stat, p_value = stats.shapiro(grupo_data)
    es_normal = p_value > 0.05
    if not es_normal:
        normalidad_ok = False
    
    print(f"Grupo {grupo}:")
    print(f"  Estadístico W: {stat:.4f}")
    print(f"  p-valor: {p_value:.4f}")
    print(f"  {'✓ Normal' if es_normal else '✗ No normal'} (α = 0.05)\n")

print(f"Supuesto de normalidad: {'✓ CUMPLIDO' if normalidad_ok else '✗ VIOLADO'}")
```

#### 4.2 Prueba de Homogeneidad de Varianzas

```{pyodide}
print("\n2. HOMOGENEIDAD DE VARIANZAS (Prueba de Levene):")
print("H₀: Las varianzas son iguales en todos los grupos")
print("H₁: Al menos una varianza es diferente\n")

# Separar datos por grupo
grupos = [my_data[my_data['group'] == grupo]['weight'] 
          for grupo in my_data['group'].cat.categories]

# Prueba de Levene
stat_levene, p_levene = stats.levene(*grupos)
homogeneidad_ok = p_levene > 0.05

print(f"Estadístico de Levene: {stat_levene:.4f}")
print(f"p-valor: {p_levene:.4f}")
print(f"{'✓ Varianzas homogéneas' if homogeneidad_ok else '✗ Varianzas heterogéneas'} (α = 0.05)")

print(f"\nSupuesto de homogeneidad: {'✓ CUMPLIDO' if homogeneidad_ok else '✗ VIOLADO'}")
```

#### 4.3 Gráficos de Diagnóstico

```{pyodide}
# Realizar ANOVA preliminar para obtener residuales
model = ols('weight ~ C(group)', data=my_data).fit()
residuales = model.resid
valores_ajustados = model.fittedvalues

# Gráficos de diagnóstico
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Q-Q plot para normalidad de residuales
stats.probplot(residuales, dist="norm", plot=axes[0])
axes[0].set_title('Q-Q Plot de Residuales')
axes[0].grid(True, alpha=0.3)

# Residuales vs valores ajustados (homogeneidad)
axes[1].scatter(valores_ajustados, residuales, alpha=0.7)
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_xlabel('Valores Ajustados')
axes[1].set_ylabel('Residuales')
axes[1].set_title('Residuales vs Valores Ajustados')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
```

### 5. ANOVA de Una Vía

```{pyodide}
print("\nANOVA DE UNA VÍA")
print("="*40)

# Método 1: scipy.stats.f_oneway
f_stat, p_value = f_oneway(*grupos)

print("Resultados del ANOVA:")
print(f"Estadístico F: {f_stat:.4f}")
print(f"p-valor: {p_value:.4f}")
print(f"Grados de libertad: {len(grupos)-1}, {len(my_data)-len(grupos)}")

# Método 2: usando statsmodels para tabla completa
model = ols('weight ~ C(group)', data=my_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(f"\nTabla ANOVA completa:")
print(anova_table)
```

### 6. Interpretación de Resultados

```{pyodide}
alpha = 0.05
print(f"\nINTERPRETACIÓN DE RESULTADOS:")
print("="*40)

if p_value < alpha:
    print(f"✓ SIGNIFICATIVO:")
    print(f"p-valor ({p_value:.4f}) < α ({alpha})")
    print("Rechazamos H₀: Hay evidencia significativa de diferencias entre grupos.")
    print("Al menos una media es significativamente diferente de las otras.")
    hacer_posthoc = True
else:
    print(f"✗ NO SIGNIFICATIVO:")
    print(f"p-valor ({p_value:.4f}) ≥ α ({alpha})")
    print("No rechazamos H₀: No hay evidencia suficiente de diferencias entre grupos.")
    print("Las medias no son significativamente diferentes.")
    hacer_posthoc = False

# Calcular tamaño del efecto (eta cuadrado)
ss_between = anova_table['sum_sq'][0]
ss_total = anova_table['sum_sq'].sum()
eta_squared = ss_between / ss_total

print(f"\nTamaño del efecto (η²): {eta_squared:.4f}")
if eta_squared < 0.01:
    interpretacion_eta = "efecto muy pequeño"
elif eta_squared < 0.06:
    interpretacion_eta = "efecto pequeño"
elif eta_squared < 0.14:
    interpretacion_eta = "efecto mediano"
else:
    interpretacion_eta = "efecto grande"

print(f"Interpretación: {interpretacion_eta}")
```

### 7. Comparaciones Post-Hoc (si es necesario)

```{pyodide}
if hacer_posthoc:
    print(f"\nCOMPARACIONES POST-HOC")
    print("="*40)
    
    # Prueba de Tukey HSD
    print("Prueba de Tukey HSD:")
    tukey_result = pairwise_tukeyhsd(endog=my_data['weight'], 
                                    groups=my_data['group'], 
                                    alpha=0.05)
    print(tukey_result)
    
    # Comparaciones por pares con t-test y corrección de Bonferroni
    print(f"\nComparaciones por pares (t-test con corrección de Bonferroni):")
    from scipy.stats import ttest_ind
    from itertools import combinations
    
    grupos_nombres = my_data['group'].cat.categories
    grupos_datos = {nombre: my_data[my_data['group'] == nombre]['weight'] 
                   for nombre in grupos_nombres}
    
    comparaciones = list(combinations(grupos_nombres, 2))
    n_comparaciones = len(comparaciones)
    alpha_corregido = 0.05 / n_comparaciones
    
    print(f"α corregido (Bonferroni): {alpha_corregido:.4f}")
    
    for grupo1, grupo2 in comparaciones:
        t_stat, p_val = ttest_ind(grupos_datos[grupo1], grupos_datos[grupo2])
        significativo = p_val < alpha_corregido
        
        media1 = grupos_datos[grupo1].mean()
        media2 = grupos_datos[grupo2].mean()
        diferencia = media1 - media2
        
        print(f"\n{grupo1} vs {grupo2}:")
        print(f"  Diferencia de medias: {diferencia:.4f}")
        print(f"  t-statistic: {t_stat:.4f}")
        print(f"  p-valor: {p_val:.4f}")
        print(f"  {'✓ Significativo' if significativo else '✗ No significativo'}")
```

### 8. Visualización Final con Resultados

```{pyodide}
# Gráfico final con resultados estadísticos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Boxplot con información del ANOVA
sns.boxplot(x='group', y='weight', data=my_data, 
            palette=['#00AFBB', '#E7B800', '#FC4E07'], ax=ax1)
sns.stripplot(x='group', y='weight', data=my_data, 
              color='black', alpha=0.6, ax=ax1)

ax1.set_title(f'ANOVA Una Vía\nF = {f_stat:.3f}, p = {p_value:.4f}')
ax1.set_xlabel('Tratamiento')
ax1.set_ylabel('Peso')

# Agregar líneas horizontales para mostrar medias
for i, grupo in enumerate(my_data['group'].cat.categories):
    media = my_data[my_data['group'] == grupo]['weight'].mean()
    ax1.hlines(media, i-0.4, i+0.4, colors='red', linestyles='--', linewidth=2)

# Gráfico de medias con intervalos de confianza
medias = my_data.groupby('group')['weight'].mean()
errores = my_data.groupby('group')['weight'].sem() * 1.96  # 95% CI

x_pos = range(len(medias))
bars = ax2.bar(x_pos, medias, yerr=errores, capsize=5, 
               color=['#00AFBB', '#E7B800', '#FC4E07'], alpha=0.7,
               error_kw={'elinewidth': 2, 'capthick': 2})

ax2.set_xticks(x_pos)
ax2.set_xticklabels(medias.index)
ax2.set_title('Medias con Intervalos de Confianza (95%)')
ax2.set_xlabel('Tratamiento')
ax2.set_ylabel('Peso Promedio')

# Agregar valores en las barras
for i, (bar, media) in enumerate(zip(bars, medias)):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + errores.iloc[i],
             f'{media:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()
```

### 9. Comparación con Alternativa No Paramétrica

```{pyodide}
# Comparar con Kruskal-Wallis (alternativa no paramétrica)
from scipy.stats import kruskal

h_stat, p_kruskal = kruskal(*grupos)

print(f"\nCOMPARACIÓN CON ALTERNATIVA NO PARAMÉTRICA:")
print("="*50)
print(f"ANOVA paramétrico:    F = {f_stat:.4f}, p = {p_value:.4f}")
print(f"Kruskal-Wallis:       H = {h_stat:.4f}, p = {p_kruskal:.4f}")

if (p_value < 0.05) == (p_kruskal < 0.05):
    print("✓ Ambos tests concuerdan en la conclusión")
else:
    print("⚠ Los tests difieren - revisar supuestos del ANOVA")
```

## Cuándo Usar ANOVA vs Alternativas

**Usa ANOVA cuando:**
- Los supuestos se cumplen (normalidad y homogeneidad)
- Quieres mayor poder estadístico
- Los tamaños de muestra son adecuados

**Usa Kruskal-Wallis cuando:**
- Los supuestos del ANOVA se violan
- Los datos son ordinales
- Hay valores atípicos extremos

**Usa transformaciones cuando:**
- Los supuestos se violan ligeramente
- La transformación mejora la normalidad/homogeneidad

## Resumen de la Metodología

1. **Exploración**: Estadísticas descriptivas y visualización
2. **Verificación de supuestos**: Normalidad (Shapiro-Wilk) y homogeneidad (Levene)
3. **ANOVA**: Calcular estadístico F y p-valor
4. **Interpretación**: Decidir sobre H₀ y calcular tamaño del efecto
5. **Post-hoc**: Si es significativo, identificar qué grupos difieren
6. **Validación**: Comparar con alternativas no paramétricas si es necesario